In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="64e32be1-b77c-4786-8dab-266d4a890995")

In [ ]:
index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [ ]:
# load the data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, "*.pdf", loader_cls=PyPDFLoader)
    documents =  loader.load()
    return documents

In [ ]:
# extract the data
extracted_data = load_pdf("data/")



In [ ]:
# converting corpus into chunks
def text_split(extracted_data):
    splitted_text = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    chunks = splitted_text.split_documents(extracted_data)

    return chunks

In [ ]:
chunks = text_split(extracted_data)
len(chunks)
chunks

In [ ]:
# converting the chunks into vector embeddings
# first get the embedding model
# chunk are of type document and are in the form of list, so we need to convert them into string

from sentence_transformers import SentenceTransformer

def convert_chunks(chunks):
    chunks = [str(chunk) for chunk in chunks]
    return chunks

chunks = convert_chunks(chunks)


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(chunks)
print(embeddings)


In [ ]:
chunks